In [1]:
from pypinyin import pinyin, lazy_pinyin, Style
from enum import Enum
import random
import time
class SelectMode(Enum):
    ALL = 1,
    FIRST = 2,
    MANUAL = 3,
    
def getPyList(wordList, style=Style.TONE3, selectMode = SelectMode.MANUAL):
    pyList = pinyin(wordList, style=style, heteronym=True)

    for word, py, i in zip(wordList, pyList, range(len(wordList))):
        if len(py) > 1:
            if selectMode == SelectMode.FIRST:
                pyList[i] = [py[0]]
            elif selectMode == SelectMode.MANUAL:
                idxs = [int(idx) - 1 for idx in input("The id of {} - {}: ".format(word, py)).split(',')]
                pyList[i] = [py[idx] for idx in idxs]
    #print(list(zip(wordList,pyList)))
    return dict(zip(wordList, pyList))

def print2(words):
    newWords = []
    for word in words:
        if word == '~':
            newWords.append('({})'.format(getPyList(newWords[-1], Style.TONE, SelectMode.FIRST)[newWords[-1]][0]))
        else:
            newWords.append(word)
    print("".join(newWords))

def dedupWordLists(wordLists):
    newWordLists = []
    allwords = set()
    for wordList in wordLists:
        dedupedWordList = list(set(filter(lambda x: x not in allwords, wordList)))
        newWordLists.append("".join(dedupedWordList))
        allwords.update(dedupedWordList)
    return newWordLists
        
def printResult(right_count, total_count):
    if total_count > 0:
        print2("你完~成~了{}道~题~，答对了{}道~题~，得~分~是：{}".format(total_count, right_count, int(100.0*right_count/total_count)))
        print()
        if right_count == total_count:
            return 1
    print()
    return 0

def writePy(wordDict):
    print2("谦谦你好，今天我们要学习下面的汉字。")
    i = 0
    items = list(wordDict.items())
    while i < len(items):
        print(" ".join(["{}({})".format(word, py[0]) for word, py in items[i: i+5]]))
        i += 5
        
    print2("请~你按~照~正确~的拼音写一遍~吧")
    for word, py in sorted(items, key = lambda x: random.random()):
        while 1: 
            py2 = input("{} 的拼音是：{}  ".format(word, py[0]))
            if py2 == '#':
                print2("结~束~学习")
                print()
                return
            
            if py2 == py[0]:
                break
            else:
                print2("写错~了 :(, 再写一次吧")
    print()

def answerPy(wordDict):
    right_count = 0
    total_count = 0
    print2("谦谦你好，请~你根~据~给~定~的汉字写正~确~的拼音。")
    for word, py in sorted(wordDict.items(), key = lambda x: random.random()):
        total_count += 1
        i = 0
        while i < 5: 
            py2 = input("{}的拼音是： ".format(word))
            if py2 == '#':
                print2("结~束~练~习")
                printResult(right_count, total_count)
                print()
                return
            
            if py2.lower() in py:
                print2("恭~喜~你答~对了！")
                right_count += 1                
                break
            else:
                print2("答~错~了 :(, 再~试~一次~吧")
                i += 1
        if i == 5:
            print2("很~遗~憾~，你没有答~对，去问问你的爸爸妈妈吧...")
        if total_count == 10:
            break
    return printResult(right_count, total_count)
    
    
def chooseWord(wordDict, wordLists):
    right_count = 0
    total_count = 0
    print2("谦谦你好，请~你根~据~拼音选~择~正~确~的汉字。")
    for word, py in sorted(wordDict.items(), key = lambda x: random.random()):
        total_count += 1
        for wordList in wordLists:
            if word in wordList:
                wordList2 = sorted(wordList, key = lambda x: random.random())[:4]
                if word not in wordList2:
                    replaceIdx = random.randint(0, 3)
                    wordList2[replaceIdx] = word
                label = -1
                for i in range(len(wordList2)):
                    if wordList2[i] == word:
                        label = i
                if label < 0:
                    print("Something is wrong, please check!")
                    total_count -= 1
                    break
                print("下面哪个汉字的拼音是 {}:".format(py[0]))
                for idx, word in zip("ABCD", wordList2):
                    print("{}. {}".format(idx, word))
                i = 0
                while i < 3: 
                    choice = input("请选择:  ")
                    if choice == '#':
                        print2("结~束~练~习")
                        printResult(right_count, total_count)
                        print()
                        return
                    if choice.lower() < 'a' or choice.lower() > 'd':
                        continue
                    if ord(choice.lower()) - ord('a') == label:
                        print2("恭~喜~你答~对了！")
                        right_count += 1
                        break
                    else:
                        print2("答~错~了 :(, 再~试~一次~吧")
                        i += 1
                if i == 3:
                    print2("很~遗~憾~，你没有答~对，去问问你的爸爸妈妈吧...")
        if total_count == 10:
            break

    return printResult(right_count, total_count)
    
def isCorrect(wordDict, wordLists):
    right_count = 0
    total_count = 0
    print2("谦谦你好，请~你判~断~下面~的拼音对不对。如果正~确~请~填~写Y，如果错~误~请~填写N。")
    
    for word, py in sorted(wordDict.items(), key = lambda x: random.random()):
        total_count += 1
        for wordList in wordLists:
            if word in wordList:
                py2 = py[0]
                if random.random() < 0.5 + 1.0/len(wordList):
                    py2 = random.choice([wordDict[w][0] for w in wordList])
                while 1:
                    ans = input("汉字 {} 的拼音是 {}，对吗?  ".format(word, py2))
                    if ans == '#':
                        print2("结~束~练~习")
                        printResult(right_count, total_count)
                        print()
                        return
                    if ans.lower() != 'y' and ans.lower() != 'n':
                        continue
                    if (ans.lower() == 'y') == (py2 == py[0]):
                        print2("恭~喜~你答~对了！")
                        right_count += 1
                        break
                    else:
                        print2("很~遗~憾~，你没有答~对 :(")
                        break
        if total_count == 10:
            break

    return printResult(right_count, total_count)

def merge1(wordLists, selectMode):
    wordLists = dedupWordLists(wordLists)
    wordDict = getPyList("".join(wordLists), selectMode = selectMode)
    #print(wordDict)
    writePy(wordDict)
    print2("学习结~束~，下面~开始做~题~目~了。")
    print()
    print()
    isCorrect(wordDict, wordLists)
    chooseWord(wordDict, wordLists)
    answerPy(wordDict)
    
def merge2(wordLists, selectMode, isWritePy = True):
    num_stars = 0
    
    wordLists = dedupWordLists(wordLists)
    wordDict = getPyList("".join(wordLists), selectMode = selectMode)
    #print(wordDict)
    if isWritePy:
        writePy(wordDict)
    else:
        print2("谦谦你好，你需~要~先看~看今天学什么吗？")
        time.sleep(0.01)
        print()
        ans = input("需要请输入Y，不需要请输入N   ")
        if ans.lower() == 'y':
            writePy(wordDict)
        print2("学习结~束~，下面~开始做~题~目~了。")
        print(20*"\n")
        while 1:
            print2("请~选~择~题~目~类~型~：")
            print2("1. 根~据~拼音选~择~汉字")
            print2("2. 判~断~拼音是否~正~确~")
            print2("3. 根~据~汉字填~写~拼音")
            print2("4. 结~束~练~习~")
            time.sleep(0.1)
            ans = input("你的选择是  ")
            print()
            print()
            if ans == '1':
                num_stars += chooseWord(wordDict, wordLists)
            elif ans == '2':
                num_stars += isCorrect(wordDict, wordLists)
            elif ans == '3':
                num_stars += answerPy(wordDict)
            elif ans == '4':
                print2("谦谦，今天的学习结~束~了。")
                print2("你今天总~共~得~到了{}个100分，让你的爸爸给你{}个星~星吧".format(num_stars, num_stars))
                print2("欢~迎~你明天继~续~回来学习拼音，再见！")
                break
            else:
                continue

In [2]:
wordLists = [
    "包吃要鱼分又友出来花叶草山上",
]
merge2(wordLists, SelectMode.FIRST, False)

谦谦你好，你需(xū)要(yào)先看(kàn)看今天学什么吗？

需要请输入Y，不需要请输入N   y
谦谦你好，今天我们要学习下面的汉字。
要(yao4) 花(hua1) 分(fen1) 草(cao3) 包(bao1)
来(lai2) 山(shan1) 上(shang4) 出(chu1) 鱼(yu2)
又(you4) 友(you3) 吃(chi1) 叶(ye4)
请(qǐng)你按(àn)照(zhào)正确(què)的拼音写一遍(biàn)吧
上 的拼音是：shang4  shang4
友 的拼音是：you3  you3
山 的拼音是：shan1  shan1
分 的拼音是：fen1  fen1
来 的拼音是：lai2  lai2
花 的拼音是：hua1  hua1
包 的拼音是：bao1  bao1
草 的拼音是：cao3  cao3
叶 的拼音是：ye4  ye4
鱼 的拼音是：yu2  yu2
又 的拼音是：you4  you4
吃 的拼音是：chi1  chi1
要 的拼音是：yao4  yao4
出 的拼音是：chu1  chu1

学习结(jié)束(shù)，下面(miàn)开始做(zuò)题(tí)目(mù)了。





















请(qǐng)选(xuǎn)择(zé)题(tí)目(mù)类(lèi)型(xíng)：
1. 根(gēn)据(jù)拼音选(xuǎn)择(zé)汉字
2. 判(pàn)断(duàn)拼音是否(fǒu)正(zhèng)确(què)
3. 根(gēn)据(jù)汉字填(tián)写(xiě)拼音
4. 结(jié)束(shù)练(liàn)习(xí)
你的选择是  1


谦谦你好，请(qǐng)你根(gēn)据(jù)拼音选(xuǎn)择(zé)正(zhèng)确(què)的汉字。
下面哪个汉字的拼音是 shang4:
A. 上
B. 又
C. 吃
D. 叶
请选择:  a
恭(gōng)喜(xǐ)你答(dá)对了！
下面哪个汉字的拼音是 yu2:
A. 草
B. 吃
C. 来
D. 鱼
请选择:  d
恭(gōng)喜(xǐ)你答(dá)对了！
下面哪个汉字的拼音是 you4:
A. 来
B. 又
C. 鱼
D. 包
请选择:  b
恭(gōng)喜(xǐ)你答(dá)对了！
下面哪个汉字的拼音